In [1]:
# _____________ read from SQLite and write to Azure
# connect to current source SQLite
import sqlite3
import pandas as pd
conn = sqlite3.connect(r"D:\NMT\OneDrive\Viettravel Airline\Database\VTA_RM.db")
c = conn.cursor()

In [5]:
payment_detail = pd.read_sql_query('SELECT * FROM payment_detail', conn)
payment_detail['modified_time'] = pd.to_datetime(payment_detail['modified_time'])
payment_detail['BOOK_DATE_GMT'] = pd.to_datetime(payment_detail['BOOK_DATE_GMT'])
payment_detail['BOOK_DATE_LCL'] = pd.to_datetime(payment_detail['BOOK_DATE_LCL'])
payment_detail['LAST_MODIFIED_GMT'] = pd.to_datetime(payment_detail['LAST_MODIFIED_GMT'])
payment_detail['LAST_MODIFIED_LCL'] = pd.to_datetime(payment_detail['LAST_MODIFIED_LCL'])
payment_detail['DATE_PAID_GMT'] = pd.to_datetime(payment_detail['DATE_PAID_GMT'])
payment_detail['DATE_PAID_LCL'] = pd.to_datetime(payment_detail['DATE_PAID_LCL'])

In [4]:
payment_detail.iloc[:, 30:40].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380850 entries, 0 to 380849
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   VCHR_CREATED_GMT  456 non-null     object
 1   VCHR_CREATED_LCL  456 non-null     object
 2   RES_CURRENCY      380772 non-null  object
 3   CURRENCY_PAID     380772 non-null  object
 4   AMOUNT_PAID       380772 non-null  object
 5   BASE_CURRENCY     380772 non-null  object
 6   BASE_AMOUNT       380772 non-null  object
 7   RPT_CURRENCY      380772 non-null  object
 8   RPT_AMOUNT        380772 non-null  object
 9   file_name         380850 non-null  object
dtypes: object(10)
memory usage: 29.1+ MB


In [5]:
payment_detail.iloc[:, 30:40].head()

,VCHR_CREATED_GMT,VCHR_CREATED_LCL,RES_CURRENCY,CURRENCY_PAID,AMOUNT_PAID,BASE_CURRENCY,BASE_AMOUNT,RPT_CURRENCY,RPT_AMOUNT,file_name
0,None,None,VND,VND,"664,630.00",VND,"664,630.00",VND,"664,630.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
1,None,None,VND,VND,"-664,630.00",VND,"-664,630.00",VND,"-664,630.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
2,None,None,VND,VND,"-15,186,000.00",VND,"-15,186,000.00",VND,"-15,186,000.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
3,None,None,VND,VND,"-1,089,000.00",VND,"-1,089,000.00",VND,"-1,089,000.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
4,None,None,VND,VND,"-423,500.00",VND,"-423,500.00",VND,"-423,500.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...


In [21]:
df = payment_detail[['RPT_AMOUNT', 'file_name']].copy()

tmp = payment_detail['RPT_AMOUNT']
df['RPT_AMOUNT'].str.replace(',', '')
df['RPT_AMOUNT'].fillna(tmp, inplace=True)
df

,RPT_AMOUNT,file_name
0,"664,630.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
1,"-664,630.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
2,"-15,186,000.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
3,"-1,089,000.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
4,"-423,500.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
...,...,...
382830,1456080,ACCOUNTING_003_PAYMENT_DETAIL - 20220921.csv
382831,1747680,ACCOUNTING_003_PAYMENT_DETAIL - 20220921.csv
382832,755040,ACCOUNTING_003_PAYMENT_DETAIL - 20220921.csv
382833,873840,ACCOUNTING_003_PAYMENT_DETAIL - 20220921.csv


In [24]:
def file_name_modified(path:str):
    all_dir_files = glob.glob(path + '/*')
    files = pd.DataFrame()
    m_time = []

    for dir_file in all_dir_files:

        # file modification timestamp of a file
        f_m_time = os.path.getmtime(dir_file)
        # convert timestamp into DateTime object
        f_dt_m = datetime.datetime.fromtimestamp(f_m_time)
        m_time += [f_dt_m]

    files_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -1]
    table_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -2]
    return pd.DataFrame({'file_name':files_name, 'modified_time':m_time,
                         'table_name':table_name, 'dir_file':all_dir_files})

In [ ]:

cargo = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\cargo'
flown_aircraft_leg = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\flown_aircraft_leg'
inflow_cash = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\inflow_cash'
pax_revenue = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\pax_revenue'
payment_detail = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\payment_detail'
reservation = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\reservation'

dim_agent = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_agent'
dim_calendar = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_calendar'
dim_fare_code = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_fare_code'
dim_routes = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_routes'
dim_slot_time = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_slot_time'
flight_type = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\flight_type'


dir_list = [cargo, flown_aircraft_leg, inflow_cash, pax_revenue, payment_detail, reservation,
           dim_agent, dim_calendar, dim_routes, dim_fare_code, dim_slot_time, flight_type]
frame_list = []

for dir_x in dir_list:
    frame_list.append(file_name_modified(dir_x))

new_log_table = pd.concat(frame_list, axis=0, ignore_index=True)

query = """
SELECT *
    FROM log_table
"""

previous_log_table = pd.read_sql_query(query, conn)
previous_log_table['modified_time'] = pd.to_datetime(previous_log_table['modified_time'])

diff_table = pd.concat([previous_log_table, new_log_table]).drop_duplicates(ignore_index=True, keep=False)
keep_table = pd.concat([previous_log_table, diff_table, diff_table]).drop_duplicates(ignore_index=True, keep=False)
remove_table = pd.concat([previous_log_table, keep_table]).drop_duplicates(ignore_index=True, keep=False)
add_table = pd.concat([new_log_table, keep_table]).drop_duplicates(ignore_index=True, keep=False)

In [54]:
import tqdm
import glob

all_dir = list(glob.glob(r'C:\Users\admin\Desktop\New folder' + '/*'))

li = []
for i in all_dir:
    df = pd.read_csv(i, skiprows=2)
    df.dropna(how='all', inplace=True)
    li.append(df)
add_payment_detail = pd.concat(li, axis=0, ignore_index=True)

#     drop the 'unamed 39' column if exists
if len(add_payment_detail.columns) == 40:
    add_payment_detail.drop(add_payment_detail.columns[-1], axis=1, inplace=True)

#         rename columns
add_payment_detail.columns = ['CONFIRMATION_NUM', 'BOOKING_TYPE', 'RES_STATUS',
    'RES_PAYMENT_ID', 'BOOK_DATE_GMT', 'BOOK_DATE_LCL', 'LAST_MODIFIED_GMT',
    'LAST_MODIFIED_LCL', 'BOOKING_AGENT', 'CRS_CODE', 'USER_ID',
    'DEPT_NAME', 'DESCRIPTION', 'IATA_NUM', 'PARENT_IATA_NUM',
    'PERSON_ORG_ID', 'FIRST_NAME', 'LAST_NAME', 'CARDHOLDER_NAME',
    'DATE_PAID_GMT', 'DATE_PAID_LCL', 'ACCOUNT_NUM', 'PAYMENT_TYPE',
    'PAYMENT_METHOD', 'PAYMENT_METHOD_DESC', 'TRANS_STATUS', 'REFERENCE',
    'AUTHORIZATION_CODE', 'VOUCHER_NUM', 'VOUCHER_NUM_FULL',
    'VCHR_CREATED_GMT', 'VCHR_CREATED_LCL', 'RES_CURRENCY', 'CURRENCY_PAID',
    'AMOUNT_PAID', 'BASE_CURRENCY', 'BASE_AMOUNT', 'RPT_CURRENCY',
    'RPT_AMOUNT']
add_payment_detail['IATA_NUM'].fillna(add_payment_detail['USER_ID'], inplace=True)
add_payment_detail['IATA_NUM'].fillna(add_payment_detail['BOOKING_AGENT'], inplace=True)
try:
    tmp = add_payment_detail['RPT_AMOUNT'].copy()
    add_payment_detail['RPT_AMOUNT'] = add_payment_detail['RPT_AMOUNT'].str.replace(',', '')
    add_payment_detail['RPT_AMOUNT'].fillna(tmp, inplace=True)
except:
    pass

In [55]:
add_payment_detail['RPT_AMOUNT']

0            664630.00
1           -664630.00
2         -15186000.00
3          -1089000.00
4           -423500.00
              ...     
382830       1456080.0
382831       1747680.0
382832        755040.0
382833        873840.0
382834       3163680.0
Name: RPT_AMOUNT, Length: 382835, dtype: object